## Tratamento do PPA por regionalização

Nesse notebook são tratados os dados do Plano Plurianual por regionalização (2022 - 2025) da Prefeitura Municipal de São Paulo

In [1]:
import pandas as pd
from utils.load_csv import load_csv
from utils.save_csv import save_csv

In [2]:
df_ppa_reg = load_csv("ppa_reg.csv")

#### Criação "id"

Cria variável "id" concatenando variáveis de código da base de dados para "chave" do merge

In [3]:
df_ppa_reg["id"] = df_ppa_reg[["codigo orgao",
                               "codigo unidade",
                               "codigo programa",
                               "codigo acao"]].apply(lambda x: "_".join(x.astype(str)), axis = 1)

In [4]:
cols = ["id"] + [col for col in df_ppa_reg.columns if col != "id"]

df_ppa_reg = df_ppa_reg[cols]

#### Transformação das variáveis de planejamento orçamentário

Transforma as variáveis de planejamento orçamentário da base de dados de string para float

In [5]:
cols_plan_recurso = ["valor 2022",
                     "valor 2023",
                     "valor 2024",
                     "valor 2025"]

In [6]:
for col in cols_plan_recurso:
    df_ppa_reg[col] = (df_ppa_reg[col]
                         .str.replace(".", "")
                         .str.replace(",", ".")
                         .astype(float))

#### Filtro regionalizável

Filtra a base de dados para conter apenas despesas regionalizáveis

In [7]:
supra_subprefs = ["Supra Subprefeitura",
                  "Supra Subprefeitura Leste",
                  "Supra Subprefeitura Oeste",
                  "Supra Subprefeitura Norte",
                  "Supra Subprefeitura Sul",
                  "Supra Subprefeitura Centro"]

In [8]:
df_ppa_reg = df_ppa_reg[~df_ppa_reg["descricao prefeitura regional"].isin(supra_subprefs)]

#### Normalização das subprefeituras

Trata os nomes das subprefreituras na base de dados para estar igual aos nomes do shapefile das subprefeituras

In [9]:
mapper = {"Subprefeitura Sé": "SE",
          "Subprefeitura Vila Mariana": "VILA MARIANA",
          "Subprefeitura Santana/Tucuruvi": "SANTANA-TUCURUVI",
          "Subprefeitura Campo Limpo": "CAMPO LIMPO",
          "Subprefeitura Itaim Paulista": "ITAIM PAULISTA",
          "Subprefeitura Pirituba/Jaraguá": "PIRITUBA-JARAGUA",
          "Subprefeitura Sapopemba": "SAPOPEMBA",
          "Subprefeitura Capela do Socorro": "CAPELA DO SOCORRO",
          "Subprefeitura Lapa": "LAPA",
          "Subprefeitura Jaçanã/Tremembé": "JACANA-TREMEMBE",
          "Subprefeitura Penha": "PENHA",
          "Subprefeitura São Miguel Paulista": "SAO MIGUEL",
          "Subprefeitura Perus/Anhanguera": "PERUS",
          "Subprefeitura Butantã": "BUTANTA",
          "Subprefeitura de Guaianases": "GUAIANASES",
          "Subprefeitura São Mateus": "SAO MATEUS",
          "Subprefeitura Cidade Ademar": "CIDADE ADEMAR",
          "Subprefeitura de Vila Prudente": "VILA PRUDENTE",
          "Subprefeitura Ipiranga": "IPIRANGA",
          "Subprefeitura Pinheiros": "PINHEIROS",
          "Subprefeitura Casa Verde/Cachoeirinha": "CASA VERDE-CACHOEIRINHA",
          "Subprefeitura Freguesia/Brasilândia": "FREGUESIA-BRASILANDIA",
          "Subprefeitura M'Boi Mirim": "M BOI MIRIM",
          "Subprefeitura Vila Maria/Vila Guilherme": "VILA MARIA-VILA GUILHERME",
          "Subprefeitura Aricanduva/Formosa/Carrão": "ARICANDUVA-FORMOSA-CARRAO",
          "Subprefeitura Santo Amaro": "SANTO AMARO",
          "Subprefeitura Mooca": "MOOCA",
          "Subprefeitura Ermelino Matarazzo": "ERMELINO MATARAZZO",
          "Subprefeitura Cidade Tiradentes": "CIDADE TIRADENTES",
          "Subprefeitura Itaquera": "ITAQUERA",
          "Subprefeitura Jabaquara": "JABAQUARA",
          "Subprefeitura Parelheiros": "PARELHEIROS"}

In [10]:
df_ppa_reg.loc[:, "descricao prefeitura regional"] = df_ppa_reg["descricao prefeitura regional"].map(mapper)

#### Salvar

In [11]:
pd.set_option("display.max_columns", None)

df_ppa_reg.sample(10)

,id,codigo orgao,descricao orgao,codigo unidade,descricao unidade,codigo programa,descricao programa,codigo acao,descricao acao,codigo da,descricao da,codigo meta,descricao meta,codigo regiao,descricao regiao,codigo prefeitura regional,descricao prefeitura regional,codigo distrito,descricao distrito,codigo sequencial da,codigo produto,descricao produto,descricao unidade medida,quantidade 2022,valor 2022,quantidade 2023,valor 2023,quantidade 2024,valor 2024,quantidade 2025,valor 2025
1268,16_10_3010_2832,16,Secretaria Municipal de Educação,10,Gabinete do Secretário,3010,"Manutenção, Desenvolvimento e Promoção da Qual...",2832,Manutenção e Operação dos Centros de Estudos d...,777.02.29.00.001,Subprefeitura de Vila Prudente,777,Despesa Regionalizável,2,Leste,29,VILA PRUDENTE,0,Supra-Distrital,1,2637,CEUs com CELPs implantados,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
11468,99_10_3022_6161,99,Fundo Municipal de Iluminação Pública,10,Fundo Municipal de Iluminação Pública,3022,Requalificação e Promoção da Ocupação dos Espa...,6161,Manutenção e Operação da Rede de Iluminação Pú...,777.03.11.00.001,Subprefeitura Pinheiros,777,Despesa Regionalizável,3,Oeste,11,PINHEIROS,0,Supra-Distrital,1,2977,Operação/Serviço mantido,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
9962,94_10_3005_1710,94,Fundo Especial do Meio Ambiente e Desenvolvime...,10,Fundo Especial do Meio Ambiente e Desenvolvime...,3005,Promoção da Sustentabilidade Ambiental,1710,"Ampliação, Reforma e Requalificação da UMAPAZ",777.01.02.00.001,Subprefeitura Pirituba/Jaraguá,777,Despesa Regionalizável,1,Norte,2,PIRITUBA-JARAGUA,0,Supra-Distrital,1,2773,"Intervenções para ampliação, reforma ou requal...",unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
9713,93_10_3023_6170,93,Fundo Municipal de Assistência Social,10,Fundo Municipal de Assistência Social,3023,Proteção à População em Situação de Vulnerabil...,6170,Operacionalização do Benefício de Prestação Co...,777.02.21.00.001,Subprefeitura Penha,777,Despesa Regionalizável,2,Leste,21,PENHA,0,Supra-Distrital,1,2977,Operação/Serviço mantido,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
3887,25_10_3001_2434,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,3001,Acesso à Cultura,2434,Manutenção e Operação do Centro Cultural São P...,777.02.22.00.001,Subprefeitura Ermelino Matarazzo,777,Despesa Regionalizável,2,Leste,22,ERMELINO MATARAZZO,0,Supra-Distrital,1,2977,Operação/Serviço mantido,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
3388,22_10_3009_5287,22,Secretaria Municipal de Infraestrutura Urbana ...,10,Gabinete do Secretário,3009,Melhoria da Mobilidade Urbana Universal,5287,Inspeção de Obras de Artes Especiais - OAE,777.04.12.00.001,Subprefeitura Vila Mariana,777,Despesa Regionalizável,4,Sul,12,VILA MARIANA,0,Supra-Distrital,1,2873,Inspeções especiais em Obras de Arte Especiais,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
1836,16_10_3025_2876,16,Secretaria Municipal de Educação,10,Gabinete do Secretário,3025,Proteção e Desenvolvimento Integral na Primeir...,2876,Manutenção e Operação de Centros Municipais de...,777.02.24.00.001,Subprefeitura Itaim Paulista,777,Despesa Regionalizável,2,Leste,24,ITAIM PAULISTA,0,Supra-Distrital,1,2619,CEMEIs em funcionamento,unida,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
368,12_10_3011_2358,12,Secretaria Municipal das Subprefeituras,10,Gabinete do Secretário,3011,"Modernização Tecnológica, Desburocratização e ...",2358,Manutenção e Operação de Praças de Atendimento...,777.01.07.00.001,Subprefeitura Vila Maria/Vila Guilherme,777,Despesa Regionalizável,1,Norte,7,VILA MARIA-VILA GUILHERME,0,Supra-Distrital,1,2936,Novos sistemas de informação em desenvolviment...,unida,"0,00",17.0,"0,00",17.0,0,17.0,0,17.0
8629,87_10_3009_1097,87,Fundo Municipal de Desenvolvimento de Trânsito,10,Fundo Municipal de Desenvolvimento de Trânsito,3009,Melhoria da Mobilidade Urbana Universal,1097,"Construção de Ciclovias, Ciclofaixas e Ciclorr...",777.04.18.00.001,Subprefeitura M'Boi Mirim,777,Despesa Regionalizável,4,Sul,18,M BOI MIRIM,0,Supra-Distrita

In [12]:
save_csv(df_ppa_reg, "ppa_reg.csv")

Base salva em data\ppa_reg.csv
